#### The Goodreads API allows developers access to Goodreads data.
#### The API can be used in many ways, including:these are some examples, you can find more in https://www.goodreads.com/api/index#author.show
        


In [1]:
import pandas as pd
from goodreads import client
import re
import matplotlib.pyplot as plt
import requests
import xmltodict
import json
def nulls(df):
    nulls = pd.DataFrame(df.isnull().sum().sort_values(), columns=['nulls'])
    nulls['porcentaje'] = round(100*nulls['nulls'] / len(df), 2)
    nulls.sort_values('porcentaje')
    return nulls

from json import loads, dumps
from collections import OrderedDict
from goodreads import client



API_KEY = 'shoGJW8HQemzBMKKuSOPA'

In [2]:
autores=pd.read_csv('Data/df_prueba.csv')

In [3]:
autores

,id,titulo,subtitulo,titulo_original,portada_imagen,capitulo_pdf,guia_docente,isbn,ean,isbn10,...,categoriapadre,categoriaid,categoria,es_primera_edicion,nombre_apellido,weekoftheyear,mes,tipo_precio,bins_pag,noticias
0,5543,Colores,Historia de su significado y fabricación,NaN,http://184.172.120.194/~masterli/admin/files/l...,http://184.172.120.194/~masterli/admin/files/l...,NaN,978-84-252-2289-4,9788425222894,84-252-2289-3,...,NaN,0,NaN,False,Anne Varichon,29.0,7.0,8.0,1.0,NaN
1,5322,El catalejo,NaN,NaN,http://184.172.120.194/~masterli/admin/files/l...,http://184.172.120.194/~masterli/admin/files/l...,NaN,978-84-339-3186-3,9788433931863,84-339-3186-5,...,Ficción,86,Literatura contemporánea,False,Patrick Deville,49.0,12.0,3.0,NaN,NaN
2,189016,Day to Night,NaN,NaN,http://184.172.120.194/~masterli/admin/files/l...,http://184.172.120.194/~masterli/admin/files/l...,NaN,978-3-8365-6269-0,9783836562690,3-8365-6269-3,...,Fotografía,33,Fotografía,True,Stephen Wilkes,35.0,9.0,8.0,1.0,NaN
3,5809,Los cuadernos,"Bocetos de diseñadores, ilustradores y creativos",NaN,http://184.172.120.194/~masterli/admin/files/l...,http://184.172.120.194/~masterli/admin/files/l...,NaN,978-84-9801-373-3,9788498013733,84-9801-373-9,...,NaN,0,NaN,False,Richard Brereton,31.0,8.0,7.0,1.0,NaN
4,4766,Arte de hoy,NaN,NaN,http://184.172.120.194/~masterli/admin/files/l...,http://184.172.120.194/~masterli/admin/files/l...,NaN,978-3-8228-5522-5,9783822855225,3-8228-5522-7,...,Arte,32,Arte,False,Burkhard Riemschneider,29.0,7.0,5.0,1.0,NaN


### 1. Cantidad de reviews por autor



In [4]:
autores.titulo

0          Colores
1      El catalejo
2     Day to Night
3    Los cuadernos
4      Arte de hoy
Name: titulo, dtype: object

In [8]:
version='v1'
output_filename = 'Data/{}_goodreads_autores.csv'.format(version)

# Specify the column name in your input data that contains the autors name here
column_name = "nombre_apellido"
RETURN_FULL_RESULTS = False
# Form a list of names:
nombres = autores[column_name].tolist()

In [9]:
nombres

['Anne Varichon',
 'Patrick Deville',
 'Stephen Wilkes',
 'Richard Brereton',
 'Burkhard Riemschneider']

In [10]:
def get_autor(nombres, api_key, return_full_response=False):
 
    goodreads_url = 'https://www.goodreads.com/search.xml'
    if API_KEY is not None:
        goodreads_url = goodreads_url + "?key={}".format(API_KEY)
        goodreads_url= goodreads_url + "&q={}".format(nombres)
        
    # Ping godreads for the reuslts:
    results = requests.get(goodreads_url)
    reviews_books = xmltodict.parse(results.content)['GoodreadsResponse']
    
    # if there's no results or an error, return empty results.
    if reviews_books['search'].get('total-results') == '0':
        output = {
            "id" : None,
            "books_count": None,
            "ratings_count": None,
            "text_reviews_count": None,
            "original_publication_year": None,
            "average_rating": None
        }
    else:    
        answer = reviews_books['search']['results']['work'][0]
        output = {
            "id" : answer.get('id').get('#text'),
            "books_count": answer.get('books_count').get('#text'),
            "ratings_count": answer.get('ratings_count').get('#text'),
            "text_reviews_count": answer.get('text_reviews_count').get('#text'),
            "original_publication_year": answer.get('original_publication_year').get('#text'),
            "average_rating": answer.get('average_rating') 
        }
        
    # Append some other details:    
    output['input_string'] = nombres
    output['status'] = results
    if return_full_response is True:
        output['response'] = results
    
    return output

results=[]
for nombre in nombres:
    goodreads_result = get_autor(nombre, API_KEY, return_full_response=RETURN_FULL_RESULTS)
    print(nombre)
    results.append(goodreads_result) 

pd.DataFrame(results).to_csv(output_filename, encoding='utf8')
calif_autor=pd.DataFrame(results)

Anne Varichon
Patrick Deville
Stephen Wilkes
Richard Brereton
Burkhard Riemschneider


In [11]:
calif_autor.head()

,id,books_count,ratings_count,text_reviews_count,original_publication_year,average_rating,input_string,status
0,1251386,4,49,11,2007,3.96,Anne Varichon,<Response [200]>
1,21625483,31,1115,153,2012,3.75,Patrick Deville,<Response [200]>
2,67000957,1,4,0,None,3.75,Stephen Wilkes,<Response [200]>
3,6528039,2,121,15,2009,3.97,Richard Brereton,<Response [200]>
4,547085,3,141,3,2001,3.77,Burkhard Riemschneider,<Response [200]>


### 2. Búsqueda por Titulo o ISBN

In [12]:
# Specify the column name in your input data that contains the title name here
column_name = "ean"
RETURN_FULL_RESULTS = False
# Form a list of names:
isbn = autores[column_name].head().tolist()

In [13]:
#an example
isbn=['9783836517317','9788433931863','9783836562690','9788498013733','9783822855225']

In [14]:
def get_titulo(isbn, API_KEY, return_full_response=False):
    goodsread_url = ' https://www.goodreads.com/book/isbn/'
    goodsread_url = goodsread_url + isbn + "?key={}".format(API_KEY)
    resultados = requests.get(goodsread_url)
    reviews_books = xmltodict.parse(resultados.content)
    if resultados.status_code == 404:
        output = {
        'isbn13': isbn,
        "id" : 0,
        "ratings_count": 0,
        "average_rating": 0,
        "text_reviews_count": 0,
        "original_publication_year": 0 }
    else :
        answer = reviews_books['GoodreadsResponse']['book']['work']
        output = {
		'isbn13': reviews_books['GoodreadsResponse']['book']['isbn13'] ,
		"id" : answer.get('id').get('#text')  ,
		"ratings_count": answer.get('ratings_count').get('#text') ,
		"text_reviews_count": answer.get('text_reviews_count').get('#text') ,
		"original_publication_year": answer.get('original_publication_year').get('#text') ,
		'average_rating': reviews_books['GoodreadsResponse']['book']['average_rating'] }
    
    output['input_string'] = isbn
    output['status'] = resultados
    return output

In [15]:
results=[]
for i in isbn:
    print(i)
    df=get_titulo(i, API_KEY)
    results.append(df)

9783836517317
9788433931863
9783836562690
9788498013733
9783822855225


### 3. Search by Title and author

In [16]:
def get_info_titulo(nombre_del_autor, titulo, API_KEY, return_full_response=False):
    goodsread_url = ' https://www.goodreads.com/book/title.xml/'
    goodsread_url = goodsread_url + "?author={}".format(nombre_del_autor) + "&key={}".format(API_KEY) + "&title={}".format(titulo)
    resultados = requests.get(goodsread_url)
    reviews_books = xmltodict.parse(resultados.content)#me trae un xml
    if resultados.status_code != 200:
        
        output= {'autor': nombre_del_autor ,
		"id" : 0,
        'title': titulo,
        "popular_shelves": 0,
        'similar_books':   0}
    else:
        output=reviews_books['GoodreadsResponse']['book']
        #output = {
		#'isbn': reviews_books['GoodreadsResponse']['book']['isbn13'] ,
		#"id" : reviews_books['GoodreadsResponse']['book']['id'],
        #'title': reviews_books['GoodreadsResponse']['book']['title'],
        #"popular_shelves": reviews_books['GoodreadsResponse']['book']['popular_shelves'],
        #'similar_books':   reviews_books['GoodreadsResponse']['book']['similar_books']}
    #output['input_string'] = titulo
    #output['autor'] = nombre_del_autor
    #output=pd.json_normalize(output)
    return output
   


In [17]:
# Specify the column name in your input data that contains the autors names and title here
column_name1 = "nombre_apellido"
column_name2='titulo'
RETURN_FULL_RESULTS = False
# Form a list of names:
df = autores[[column_name1, column_name2]]
df=df.drop_duplicates(keep='first')
df=df.dropna()
df=df.values.tolist()


In [18]:
df

[['Anne Varichon', 'Colores'],
 ['Patrick Deville', 'El catalejo'],
 ['Stephen Wilkes', 'Day to Night'],
 ['Richard Brereton', 'Los cuadernos'],
 ['Burkhard Riemschneider', 'Arte de hoy']]

In [19]:
results = []

# Go through each titulo in turn
for i in df :
    output = get_info_titulo(i[0], i[1],API_KEY)
    print(i[0], i[1])
    #if output['id']!='sin_dato':
    results.append(output)
results = pd.DataFrame(results)
results.to_csv('Data/resultados.csv')    

Anne Varichon Colores
Patrick Deville El catalejo
Stephen Wilkes Day to Night
Richard Brereton Los cuadernos
Burkhard Riemschneider Arte de hoy


In [20]:
results

,id,title,isbn,isbn13,asin,kindle_asin,marketplace_id,country_code,image_url,small_image_url,...,url,link,authors,reviews_widget,popular_shelves,book_links,buy_links,series_works,similar_books,autor
0,53180442,Colores: Historia de su significado y fabricación,None,None,NaN,NaN,NaN,AR,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,...,https://www.goodreads.com/book/show/53180442-c...,https://www.goodreads.com/book/show/53180442-c...,"{'author': {'id': '608527', 'name': 'Anne Vari...",<style>\n #goodreads-widget {\n font-famil...,"{'shelf': [{'@name': 'to-read', '@count': '77'...","{'book_link': {'id': '8', 'name': 'Libraries',...","{'buy_link': [{'id': '1', 'name': 'Amazon', 'l...",NaN,"{'book': [{'id': '539282', 'uri': 'kca://book/...",NaN
1,10585823,El catalejo,8433931865,9788433931863,NaN,NaN,NaN,AR,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,...,https://www.goodreads.com/book/show/10585823-e...,https://www.goodreads.com/book/show/10585823-e...,"{'author': [{'id': '219107', 'name': 'Patrick ...",<style>\n #goodreads-widget {\n font-famil...,"{'shelf': [{'@name': 'to-read', '@count': '39'...","{'book_link': {'id': '8', 'name': 'Libraries',...","{'buy_link': [{'id': '1', 'name': 'Amazon', 'l...",NaN,"{'book': [{'id': '25883848', 'uri': 'kca://boo...",NaN
2,43184141,Stephen Wilkes. Day to Night,3836562693,9783836562690,NaN,NaN,NaN,AR,https://i.gr-assets.com/images/S/compressed.ph...,https://i.gr-assets.com/images/S/compressed.ph...,...,https://www.goodreads.com/book/show/43184141-s...,https://www.goodreads.com/book/show/43184141-s...,"{'author': [{'id': '38276', 'name': 'Lyle Rexe...",<style>\n #goodreads-widget {\n font-famil...,"{'shelf': [{'@name': 'to-read', '@count': '17'...","{'book_link': {'id': '8', 'name': 'Libraries',...","{'buy_link': [{'id': '1', 'name': 'Amazon', 'l...",NaN,"{'book': [{'id': '54907902', 'uri': 'kca://boo...",NaN
3,0,Los cuadernos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,Richard Brereton
4,5671986,Arte De Hoy,3822855227,9783822855225,NaN,NaN,NaN,AR,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,...,https://www.goodreads.com/book/show/5671986-ar...,https://www.goodreads.com/book/show/5671986-ar...,"{'author': [{'id': '2538960', 'name': 'Riemsch...",<style>\n #goodreads-widget {\n font-famil...,"{'shelf': [{'@name': 'to-read', '@count': '4'}...","{'book_link': {'id': '8', 'name': 'Libraries',...","{'buy_link': [{'id': '1', 'name': 'Amazon', 'l...",NaN,NaN,NaN


#### I want to know similar books

In [21]:
similar_books = []
for i in range(len(results['id'])):
    for e in range (len(results['similar_books'])):   
        try:
            similar_books.append({ 'id': results['id'][i],
                                    'isbn13': results['isbn13'][i],
                                 'titulo': results['title'][i],
                                'similar_books_book_id': results['similar_books'][i]['book'][e]['id'],
                                'similar_books_titulo':  results['similar_books'][i]['book'][e]['title'], 
                                'similar_books_isbn13':  results['similar_books'][i]['book'][e]['isbn13']})
        except: 
            pass
        

In [22]:
similar_books=pd.DataFrame(similar_books)

In [23]:
similar_books

,id,isbn13,titulo,similar_books_book_id,similar_books_titulo,similar_books_isbn13
0,53180442,None,Colores: Historia de su significado y fabricación,539282,Fences (The Century Cycle #6),9780452264014
1,53180442,None,Colores: Historia de su significado y fabricación,95833,M. Butterfly,9780822207122
2,53180442,None,Colores: Historia de su significado y fabricación,86,The Heidi Chronicles: Uncommon Women and Other...,9780679734994
3,53180442,None,Colores: Historia de su significado y fabricación,297779,Master Harold...and the boys,9780140481877
4,53180442,None,Colores: Historia de su significado y fabricación,2403970,That Championship Season,9780822211266
5,10585823,9788433931863,El catalejo,25883848,The Hating Game,None
6,10585823,9788433931863,El catalejo,42201431,The Unhoneymooners,None
7,10585823,9788433931863,El catalejo,1154047,Sarrasine,9782253193050
8,10585823,9788433931863,El catalejo,1228779,Forever Valley,9780803289512
9,10585823,9788433931863,El catalejo,16071333,14,9782707322579


#### Popular shelves

In [24]:
listados_all=[]
for i in range(len(results['id'])):
    for e in range(100):
        try:
            listados_all.append({ 'id': results['id'][i], 
                                 'isbn13': df['isbn13'][i], 
                                 'titulo': results['title'][i],
                                 'list': results['popular_shelves'][i]['shelf'][e]['@name'],
                                 'count':  results['popular_shelves'][i]['shelf'][e]['@count']})
        except: 
            pass
        
listados_all=pd.DataFrame(listados_all)